# Instruments

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Query instruments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we filter various types of instruments.
-- 2. These can be uploaded using examples in this folder.
-- 3. We will query three different types of instrument,
--    and join the results using union.
-- ============================================================

-- Select common columns and filter by ClientInternal IDs

select DisplayName, ClientInternal, DomCcy, LusidInstrumentId
from Lusid.Instrument.Equity
where ClientInternal = 'EQ18BCC14487A54'

union

select DisplayName, ClientInternal, DomCcy, LusidInstrumentId
from Lusid.Instrument.SimpleInstrument
where ClientInternal = 'SHOPCENCI3'

union

select DisplayName, ClientInternal, DomCcy, LusidInstrumentId
from Lusid.Instrument.TermDeposit
where ClientInternal = 'TDDFFFA295'

union

select DisplayName, ClientInternal, DomCcy, LusidInstrumentId
from Lusid.Instrument.Bond
where ClientInternal = 'UKT 2 09/07/25 GB00BTHH2R79';



#### Set properties from csv

In [ ]:
%%luminesce

-- Define file name, this is the instruments file on Drive

@@file_name = select 'example_instruments.csv';

-- Load data from Drive

@instruments_data =
use Drive.Csv with @@file_name
--file=/luminesce-examples/{@@file_name}
--addFileName
enduse;

@instProperties =
select 
ClientInternal as EntityId, 
'ClientInternal' as EntityIdType, 
'Instrument' as Domain,
"Ratings" as PropertyScope, 
RatingType as PropertyCode,
Rating as Value,
"Ratings" as EntityScope
from @instruments_data;

-- Upload the transformed data into LUSID

select *
from Lusid.Property.Writer
where ToWrite = @instProperties
limit 5;



#### Upload bond instruments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we run an ETL process on some instruments.
-- 2. First, we load a XML file of bond instruments from Drive.
-- 3. Next, we transform the shape of the instrument data.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

-- Extract Bond instrument data from LUSID Drive

@instruments_data =
use Drive.Xml
--file=/luminesce-examples/instruments.xml
--nodePath=instruments/bond
--columns
BondID=bondId
Name=name
ISIN=isin
IssueDate=issueDate
Maturity=maturity
Currency=crncy
CouponRate=couponRate
Principal=principal
CouponFreq=cpnFreq
DayCountConvention=dayCountConvention
RollConvention=rollConvention
PaymentCalendars=paymentCalendars
ResetCalendars=resetCalendars
SettleDays=settleDays
ResetDays=resetDays
enduse;

-- Transform data using SQL

@instruments = 
select
Name as DisplayName,
ISIN as Isin,
Name || ' ' || ISIN as ClientInternal,
IssueDate as StartDate,
CouponRate as CouponRate,
Currency as DomCcy,
Currency as FlowConventionsCurrency,
CouponFreq as FlowConventionsPaymentFrequency,
DayCountConvention as FlowConventionsDayCountConvention,
RollConvention as FlowConventionsRollConvention,
'' as FlowConventionsPaymentCalendars,
'' as FlowConventionsResetCalendars,
SettleDays as FlowConventionsSettleDays,
ResetDays as FlowConventionsResetDays,
Principal as Principal,
Maturity as MaturityDate
from @instruments_data;

-- Upload the transformed data into LUSID

select *
from Lusid.Instrument.Bond.Writer
where ToWrite = @instruments;



#### Upload equity instruments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we run an ETL process on some instruments.
-- 2. First, we load a CSV file of instruments from Drive.
-- 3. Next, we transform the shape of the instrument data.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

-- Extract equity instrument data from LUSID Drive

@instruments_data =
use Drive.Csv
--file=/luminesce-examples/uk_instruments.csv
enduse;

-- Transform data using SQL

@equity_instruments =
select
Name as DisplayName,
ISIN as Isin,
ClientInternal as ClientInternal,
SEDOL as Sedol,
'GBP' as DomCcy
from @instruments_data;

-- Upload the transformed data into LUSID

select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments;



#### Upload future instruments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we run an ETL process on some instruments.
-- 2. First, we load a CSV file of futures instruments from Drive.
-- 3. Next, we transform the shape of the instrument data.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

-- Extract Future instrument data from LUSID Drive

@instruments_data =
use Drive.Csv
--file=/luminesce-examples/futures.csv
enduse;

-- Transform data using SQL
@instruments = 
select
-- Contract details
contract_code as ContractDetailsContractCode,
contract_month as ContractDetailsContractMonth,
contract_size as ContractDetailsContractSize,
convention as ContractDetailsConvention,
country_id as ContractDetailsCountry,
fut_name as ContractDetailsDescription,
dom_ccy as ContractDetailsDomCcy,
exchange_code as ContractDetailsExchangeCode,
exchange_name as ContractDetailsExchangeName,
ticker_step as ContractDetailsTickerStep,
unit_value as ContractDetailsUnitValue,
-- Future details
start_date as StartDate,
maturity_date as MaturityDate,
1 as Contracts,
id as ClientInternal,
fut_name as DisplayName
from @instruments_data;

-- Create future instrument
select *
from Lusid.Instrument.Future.Writer
where ToWrite = @instruments;



#### Upload fx forward

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- 1. In this query, we run an ETL process on an Fx Forward.
-- 2. First, we load a CSV file of the forward from LUSID drive.
-- 3. Next, we transform the shape of the forward data.
-- 4. Finally we upload the forward data into LUSID.
-- ===============================================================

-- Defining scope variable

@@instrumentScope = select 'luminesce-examples';

-- Load forward into table

@fx_data = 
use Drive.Csv
--file=/luminesce-examples/fx_forward.csv
enduse;

-- Define informtion to upsert from file

@fx_forward_instrument =
select  SecurityDescription as DisplayName,
        Figi as Figi,
        TradeDate as StartDate,
        SettlementDate as MaturityDate,
        Ccy as DomCcy,
        Ccy2 as FgnCcy,
        CcyAmount as DomAmount,
        -1 * Ccy2Amount as FgnAmount,
        @@instrumentScope as Scope
from @fx_data;

-- Upsert Fx Forward to LUSID

select * from lusid.Instrument.FxForward.Writer
where ToWrite=@fx_forward_instrument;


#### Upload simple instruments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we run an ETL process on some instruments.
-- 2. First, we load a Excel file of instruments from Drive.
-- 3. Next, we transform the shape of the instrument data.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

-- Extract Simple Instrument data from LUSID Drive

@instruments_data =
use Drive.Excel
--file=/luminesce-examples/instruments.xlsx
--worksheet=simple_instruments
enduse;

-- Transform data using SQL

@simple_instruments =
select
Name as DisplayName,
ClientInternal as ClientInternal,
Currency as DomCcy,
Class as AssetClass,
Type as SimpleInstrumentType
from @instruments_data;

-- Upload the transformed data into LUSID

select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @simple_instruments;



#### Upload term deposit instruments

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we run an ETL process on some instruments.
-- 2. First, we load a Excel file of instruments from Drive.
-- 3. Next, we transform the shape of the instrument data.
-- 4. Finally we upload the instrument data into LUSID.
-- ============================================================

-- Extract Term Deposit data from LUSID Drive

@term_deposit_file = use Drive.Excel
--file=/luminesce-examples/instruments.xlsx
--worksheet=term_deposits
enduse;

-- Transform data using SQL

@term_deposit_instruments = select
id as ClientInternal,
name as DisplayName,
start_date as StartDate,
mat_date as MaturityDate,
rate as Rate,
payment_freq as FlowConventionPaymentFrequency,
day_count_convention as FlowConventionDayCountConvention,
flow_convention as FlowConventionRollConvention,
payment_cals as FlowConventionPaymentCalendars,
reset_cals as FlowConventionResetCalendars,
currency as FlowConventionCurrency
from @term_deposit_file;

-- Upload the transformed data into LUSID

select *
from Lusid.Instrument.TermDeposit.Writer
where ToWrite = @term_deposit_instruments;

